In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import torch.nn.functional as F


import torchvision
from torchvision.models import resnet18, ResNet18_Weights, vgg16, VGG16_Weights
from torchvision import datasets
from torchvision import transforms

from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt


import os
import sys
from torchsummary import summary


# from google.colab import drive
# drive.mount('/content/drive')
# #Code folder path
# %cd /content/drive/My Drive/ECE591_DL_CL_PROJECT/

#device config
torch.cuda.set_device(1)#Select GPU device 1
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Get Packages
sys.path.append('./Tools')

#Network architectures
from ResNet18_CIFAR10 import ResNet18_CIFAR10_Model
from ResNet18_TINYIMAGENET import ResNet18_TINYIMAGENET_Model
from VGG16_CIFAR10 import VGG16_CIFAR10_Model
from VGG16_TINYIMAGENET import VGG16_TINYIMAGENET_Model

#Non Corrupted DataLoaders
from CIFAR10_LOADER import CIFAR10DataLoader
from TINYIMAGENET_LOADER import TINYIMAGENETDataLoader

#Non Corrupted DataLoaders
from CIFAR10C_LOADER import CIFAR10C_DataLoader #Each Corruption is a entry in the dictionary
from TINYIMAGENET_C_LOADER import TinyImagenetC_DataLoader #Each Corruption is a entry in the dictionary

#Error Metrics
from ErrorMetrics import Evaluate_Model_TOP1


# Unpruned Models

In [2]:
#Network weights
ResNet18_CIFAR10_Weights= torch.load("./Networks/ResNet18_CIFAR10_V0.pt")
ResNet18_TINYIMAGENET_Weights=torch.load("./Networks/ResNet18_TINY_IMAGENET_V0.pt")
VGG16_CIFAR10_Weights=torch.load("./Networks/VGG16_CIFAR10_V0.pt")
VGG16_TINYIMAGENET_Weights=torch.load("./Networks/VGG16_TINY_IMAGENET_V0.pt")

#Unpruned Models
ResNet18_CIFAR10_0P=ResNet18_CIFAR10_Model()
ResNet18_CIFAR10_0P.load_state_dict(ResNet18_CIFAR10_Weights)

ResNet18_TINYIMAGENET_0P=ResNet18_TINYIMAGENET_Model()
ResNet18_TINYIMAGENET_0P.load_state_dict(ResNet18_TINYIMAGENET_Weights)

VGG16_CIFAR10_0P=VGG16_CIFAR10_Model()
VGG16_CIFAR10_0P.load_state_dict(VGG16_CIFAR10_Weights)

VGG16_TINYIMAGENET_0P=VGG16_TINYIMAGENET_Model()
VGG16_TINYIMAGENET_0P.load_state_dict(VGG16_TINYIMAGENET_Weights)


<All keys matched successfully>

# Load Datasets

In [3]:
from unittest import loader


trainloader_CIFAR10, testloader_CIFAR10=CIFAR10DataLoader() #Load Data for CIFAR10
trainloader_TINYIMAGENET, testloader_TINYIMAGENET=TINYIMAGENETDataLoader() #Load Data for TINYIMAGENET
loader_CIFAR10C=CIFAR10C_DataLoader() #Load Data for CIFAR10C
loader_TinyImagenetC=TinyImagenetC_DataLoader() #Load Data for TINYIMAGENETC

Files already downloaded and verified
Files already downloaded and verified


# Evaluate Non Corrupted Data Models

In [5]:

#Test Accuracy
loss, acc, misclassified=Evaluate_Model_TOP1(ResNet18_CIFAR10_0P, device, testloader_CIFAR10) #ResNet18 on CIFAR10
print(f"ResNet18_CIFAR10_0P: Loss: {loss:.4f}, Accuracy: {acc:.2f}%\n\n")

loss, acc, misclassified=Evaluate_Model_TOP1(ResNet18_TINYIMAGENET_0P, device, testloader_TINYIMAGENET) #ResNet18 on TINYIMAGENET
print(f"ResNet18_TINYIMAGENET_0P: Loss: {loss:.4f}, Accuracy: {acc:.2f}%\n\n")

loss, acc, misclassified=Evaluate_Model_TOP1(VGG16_CIFAR10_0P, device, testloader_CIFAR10) #VGG16 on CIFAR10
print(f"VGG16_CIFAR10_0P: Loss: {loss:.4f}, Accuracy: {acc:.2f}%\n\n")

loss, acc, misclassified=Evaluate_Model_TOP1(VGG16_TINYIMAGENET_0P, device, testloader_TINYIMAGENET) #VGG16 on TINYIMAGENET
print(f"VGG16_TINYIMAGENET_0P: Loss: {loss:.4f}, Accuracy: {acc:.2f}%\n\n")


100%|██████████| 20/20 [00:01<00:00, 10.74it/s]


ResNet18_CIFAR10_0P: Loss: 0.3436, Accuracy: 89.04%




100%|██████████| 20/20 [00:02<00:00,  9.18it/s]


ResNet18_TINYIMAGENET_0P: Loss: 1.7581, Accuracy: 60.36%




100%|██████████| 20/20 [00:01<00:00, 10.43it/s]


VGG16_CIFAR10_0P: Loss: 0.3662, Accuracy: 89.09%




100%|██████████| 20/20 [00:01<00:00, 10.18it/s]

VGG16_TINYIMAGENET_0P: Loss: 2.7202, Accuracy: 54.76%




In [7]:
#Train Accuracy
loss, acc, misclassified=Evaluate_Model_TOP1(ResNet18_CIFAR10_0P, device, trainloader_CIFAR10) #ResNet18 on CIFAR10
print(f"ResNet18_CIFAR10_0P: Loss: {loss:.4f}, Accuracy: {acc:.2f}%\n\n")

loss, acc, misclassified=Evaluate_Model_TOP1(ResNet18_TINYIMAGENET_0P, device, trainloader_TINYIMAGENET) #ResNet18 on TINYIMAGENET
print(f"ResNet18_TINYIMAGENET_0P: Loss: {loss:.4f}, Accuracy: {acc:.2f}%\n\n")

loss, acc, misclassified=Evaluate_Model_TOP1(VGG16_CIFAR10_0P, device, trainloader_CIFAR10) #VGG16 on CIFAR10
print(f"VGG16_CIFAR10_0P: Loss: {loss:.4f}, Accuracy: {acc:.2f}%\n\n")

loss, acc, misclassified=Evaluate_Model_TOP1(VGG16_TINYIMAGENET_0P, device, trainloader_TINYIMAGENET) #VGG16 on TINYIMAGENET
print(f"VGG16_TINYIMAGENET_0P: Loss: {loss:.4f}, Accuracy: {acc:.2f}%\n\n")

100%|██████████| 98/98 [00:19<00:00,  4.91it/s]


ResNet18_CIFAR10_0P: Loss: 0.1349, Accuracy: 95.64%




100%|██████████| 391/391 [01:00<00:00,  6.43it/s]


ResNet18_TINYIMAGENET_0P: Loss: 0.5043, Accuracy: 87.22%




100%|██████████| 98/98 [00:19<00:00,  4.93it/s]


VGG16_CIFAR10_0P: Loss: 0.1292, Accuracy: 95.64%




100%|██████████| 391/391 [01:01<00:00,  6.38it/s]

VGG16_TINYIMAGENET_0P: Loss: 0.0141, Accuracy: 99.78%




# Evaluate Models on Corrupted Data

## CIFAR 10C

In [4]:
CIFAR10C_Corruptions=list(loader_CIFAR10C.keys())
Acc_VGG16_CIFAR10C_Corruptions={}
Acc_ResNet18_CIFAR10C_Corruptions={}

for cname in CIFAR10C_Corruptions:
    #corruption name and corruption number
    
    print(f"\nCorruption: {cname}")

    loss, acc, misclassified=Evaluate_Model_TOP1(VGG16_CIFAR10_0P, device, loader_CIFAR10C[cname]) #VGG16 on CIFAR10C
    Acc_VGG16_CIFAR10C_Corruptions[cname]=acc
    print(f"VGG16: Accuracy: {acc:.2f}%")


    loss, acc, misclassified=Evaluate_Model_TOP1(ResNet18_CIFAR10_0P, device, loader_CIFAR10C[cname]) #ResNet18 on CIFAR10
    Acc_ResNet18_CIFAR10C_Corruptions[cname]=acc  
    print(f"ResNet18: Accuracy: {acc:.2f}%")


Corruption: brightness


  0%|          | 0/196 [00:00<?, ?it/s]

100%|██████████| 196/196 [00:06<00:00, 30.99it/s]


VGG16: Accuracy: 87.52%


100%|██████████| 196/196 [00:05<00:00, 32.83it/s]


...

ResNet18: Accuracy: 67.89%

Corruption: zoom_blur


100%|██████████| 196/196 [00:06<00:00, 31.73it/s]


VGG16: Accuracy: 75.37%


100%|██████████| 196/196 [00:06<00:00, 30.58it/s]

ResNet18: Accuracy: 75.86%


## Tiny Imagenet C

In [4]:
TinyImagenetC_CorruptionClasses=list(loader_TinyImagenetC.keys())
TinyImagenetC_CorruptionLevels=list(loader_TinyImagenetC[TinyImagenetC_CorruptionClasses[0]].keys())

Acc_VGG16_TINY_IMAG_C_Corruptions={}
Acc_ResNet18_TINY_IMAG_CCorruptions={}

for cname in tqdm(TinyImagenetC_CorruptionClasses):
    print(f"\nCorruption: {cname}")
    for clevel in TinyImagenetC_CorruptionLevels:

        loss, acc, misclassified=Evaluate_Model_TOP1(VGG16_TINYIMAGENET_0P, device, loader_TinyImagenetC[cname][clevel])
        Acc_VGG16_TINY_IMAG_C_Corruptions[cname+"_"+clevel]=acc

        loss, acc, misclassified=Evaluate_Model_TOP1(ResNet18_TINYIMAGENET_0P, device, loader_TinyImagenetC[cname][clevel])
        Acc_ResNet18_TINY_IMAG_CCorruptions[cname+"_"+clevel]=acc



  0%|          | 0/19 [00:00<?, ?it/s]


Corruption: brightness


  5%|▌         | 1/19 [00:36<10:55, 36.44s/it]


Corruption: contrast


 11%|█         | 2/19 [01:12<10:13, 36.10s/it]

...

 89%|████████▉ | 17/19 [12:43<01:43, 51.77s/it]


Corruption: speckle_noise


 95%|█████████▍| 18/19 [13:40<00:53, 53.35s/it]


Corruption: zoom_blur


100%|██████████| 19/19 [14:36<00:00, 46.14s/it]


In [5]:
Acc_ResNet18_TINY_IMAG_CCorruptions


{'brightness_1': 36.08,
 'brightness_2': 34.29,
 'brightness_3': 30.52,
 'brightness_4': 25.84,
 'brightness_5': 19.87,
 'contrast_1': 21.94,
 'contrast_2': 14.91,
 'contrast_3': 7.84,
 'contrast_4': 2.56,
 'contrast_5': 1.47,
 'defocus_blur_1': 17.38,
 'defocus_blur_2': 14.24,
 'defocus_blur_3': 8.67,
 'defocus_blur_4': 4.25,
 'defocus_blur_5': 3.19,
 'elastic_transform_1': 23.21,
 'elastic_transform_2': 22.58,
 'elastic_transform_3': 18.38,
 'elastic_transform_4': 17.25,
 'elastic_transform_5': 16.83,
 'fog_1': 33.44,
 'fog_2': 28.26,
 'fog_3': 22.61,
 'fog_4': 13.41,
 'fog_5': 7.48,
 'frost_1': 31.7,
 'frost_2': 27.12,
 'frost_3': 23.49,
 'frost_4': 20.5,
 'frost_5': 17.63,
 'gaussian_blur_1': 23.87,
 'gaussian_blur_2': 12.03,
 'gaussian_blur_3': 7.36,
 'gaussian_blur_4': 5.03,
 'gaussian_blur_5': 3.95,
 'gaussian_noise_1': 34.89,
 'gaussian_noise_2': 21.8,
 'gaussian_noise_3': 11.87,
 'gaussian_noise_4': 7.52,
 'gaussian_noise_5': 5.2,
 'glass_blur_1': 38.79,
 'glass_blur_2': 17.63

In [6]:
Acc_VGG16_TINY_IMAG_C_Corruptions

{'brightness_1': 33.97,
 'brightness_2': 31.58,
 'brightness_3': 28.08,
 'brightness_4': 23.43,
 'brightness_5': 18.57,
 'contrast_1': 20.32,
 'contrast_2': 13.42,
 'contrast_3': 7.07,
 'contrast_4': 2.29,
 'contrast_5': 1.15,
 'defocus_blur_1': 19.52,
 'defocus_blur_2': 16.38,
 'defocus_blur_3': 11.84,
 'defocus_blur_4': 6.54,
 'defocus_blur_5': 5.12,
 'elastic_transform_1': 23.85,
 'elastic_transform_2': 23.62,
 'elastic_transform_3': 20.19,
 'elastic_transform_4': 18.84,
 'elastic_transform_5': 18.15,
 'fog_1': 32.26,
 'fog_2': 27.29,
 'fog_3': 21.68,
 'fog_4': 13.19,
 'fog_5': 7.74,
 'frost_1': 31.09,
 'frost_2': 27.88,
 'frost_3': 24.58,
 'frost_4': 21.77,
 'frost_5': 18.92,
 'gaussian_blur_1': 24.87,
 'gaussian_blur_2': 14.62,
 'gaussian_blur_3': 10.44,
 'gaussian_blur_4': 7.62,
 'gaussian_blur_5': 5.97,
 'gaussian_noise_1': 33.38,
 'gaussian_noise_2': 22.54,
 'gaussian_noise_3': 13.07,
 'gaussian_noise_4': 9.11,
 'gaussian_noise_5': 6.46,
 'glass_blur_1': 36.75,
 'glass_blur_2':